<a href="https://colab.research.google.com/github/ViniciusBerti/20241_maua_ecm252_intro_git/blob/main/C%C3%B3pia_de_IMT_ex_knn_dt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->




# **Exercício 3 - 3o bimestre - ECM514 Ciência de Dados**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Vinícius de Oliveira Berti, 21.01219-9' #@param {type:"string"}






# **Classificação: Knn $\times$ Árvores de Decisão**
---



In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Igor Improta, 21.00834-5' #@param {type:"string"}






# Caso: **Predição de Diagnósticos a partir de Dados de Imagens**


https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

Os dados acima são características computadas a partir de uma imagem digitalizada por *agulha fina* (PAAF) de uma massa mamária. Eles descrevem características dos núcleos celulares presentes na imagem e classificam os tumores como malignos ou benignos.







In [ ]:
#@markdown **Set Up Lab** (*run this before continue*)

import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import warnings
import os
import seaborn as sns
warnings.filterwarnings("ignore")

breast = pd.read_csv('https://github.com/Rogerio-mack/Machine-Learning-I/raw/main/data/breast.csv')

from sklearn.model_selection import train_test_split

seed = 1984
X_train, X_test, y_train, y_test = train_test_split(breast.drop(columns=['diagnosis']), breast.diagnosis, test_size=0.05, stratify=breast.diagnosis, random_state=seed)

breast = pd.concat([X_train, y_train],axis=1)
new_breast = X_test

breast.to_csv('breast.csv',index=False)

new_breast['diagnosis'] = '?'
new_breast.to_csv('new_breast.csv',index=False)

y_test.to_csv('new_breast_answers.csv',index=False)

# correlation_matrix = breast.drop(columns=['diagnosis']).corr()

# Find pairs with correlation > 0.99, excluding the diagonal
# highly_correlated_pairs = correlation_matrix[abs(correlation_matrix) > 0.99].stack()
# highly_correlated_pairs = highly_correlated_pairs[highly_correlated_pairs != 1.0]

# print("Highly correlated pairs (correlation > 0.99):")
# display(highly_correlated_pairs)


E você empregará os dados já tratados no set up acima, com os dataframes `breast` e `new_breast`. A idea aqui é empregar os dados de `breast` para estimar as classes dos novos casos desconhecidos de `new_breast`.



In [ ]:
breast.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
411,8911670,18.810,19.98,120.90,1102.0,0.08923,0.05884,0.08020,0.05843,0.1550,...,24.30,129.00,1236.0,0.12430,0.1160,0.2210,0.12940,0.2567,0.05737,M
149,9047,12.940,16.17,83.18,507.6,0.09879,0.08836,0.03296,0.02390,0.1735,...,23.02,89.69,580.9,0.11720,0.1958,0.1810,0.08388,0.3297,0.07834,B
103,859196,9.173,13.86,59.20,260.9,0.07721,0.08751,0.05988,0.02180,0.2341,...,19.23,65.59,310.1,0.09836,0.1678,0.1397,0.05087,0.3282,0.08490,B
260,864496,8.726,15.83,55.84,230.9,0.11500,0.08201,0.04132,0.01924,0.1649,...,19.62,64.48,284.4,0.17240,0.2364,0.2456,0.10500,0.2926,0.10170,B
106,91505,12.540,16.32,81.25,476.3,0.11580,0.10850,0.05928,0.03279,0.1943,...,21.40,86.67,552.0,0.15800,0.1751,0.1889,0.08411,0.3155,0.07538,B


In [ ]:
new_breast.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
274,862009,13.45,18.30,86.60,555.1,0.10220,0.08165,0.03974,0.02780,0.1638,...,25.94,97.59,699.4,0.13390,0.17510,0.13810,0.07911,0.2678,0.06603,?
107,9110127,18.03,16.85,117.50,990.0,0.08947,0.12320,0.10900,0.06254,0.1720,...,22.02,133.30,1292.0,0.12630,0.26660,0.42900,0.15350,0.2842,0.08225,?
264,90769601,11.13,16.62,70.47,381.1,0.08151,0.03834,0.01369,0.01370,0.1511,...,20.29,74.35,421.1,0.10300,0.06219,0.04580,0.04044,0.2383,0.07083,?
175,909777,10.57,18.32,66.82,340.9,0.08142,0.04462,0.01993,0.01111,0.2372,...,23.31,69.35,366.3,0.09794,0.06542,0.03986,0.02222,0.2699,0.06736,?
215,904689,12.96,18.29,84.18,525.2,0.07351,0.07899,0.04057,0.01883,0.1874,...,24.61,96.31,621.9,0.09329,0.23180,0.16040,0.06608,0.3207,0.07247,?


# Exercício. Acesse e Explore os dados

Qual o atributo classe? Quantos casos são **'B'enignos** e **'M'alignos**? Existem dados ausentes (Null ou NA)? Existem atributos **não preditores**?


# Exercício. Pré-Processamento, Conjunto de Treinamento e Teste

Prepare aqui os dados de entrada `X` e saída `y` do modelo.

Note que há um atributo além do atributo classe que não é preditor (qual?). Remova esse atributo antes de normalizar os dados. Para normalização empregue o `MinMaxScaler()` do scikit-learn (ver no material de teoria exemplo de como aplicar).

Empregue a função `train_test_split` e separe 0.3 dos dados para teste. Empregue o `random_state=42` para garantir a reprodutibilidade dos resultados e empregue o parâmetro `stratify` para garantir a mesma distribuição das classes nos conjuntos de treinamento e teste.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

X_raw = breast.drop(columns=['id', 'diagnosis'])
y = breast['diagnosis']

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.30, random_state=42, stratify=y
)

scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train_raw), columns=X_train_raw.columns, index=X_train_raw.index)
X_test  = pd.DataFrame(scaler.transform(X_test_raw),    columns=X_test_raw.columns,  index=X_test_raw.index)

xtrain_mean = X_train.to_numpy().mean()
print("X_train mean:", round(xtrain_mean, 3))

X_train mean: 0.247


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn9 = KNeighborsClassifier(n_neighbors=9)
knn9.fit(X_train, y_train)
y_pred9 = knn9.predict(X_test)
acc9 = accuracy_score(y_test, y_pred9)

knn5 = KNeighborsClassifier(n_neighbors=5)
knn5.fit(X_train, y_train)
y_pred5 = knn5.predict(X_test)
acc5 = accuracy_score(y_test, y_pred5)

print("Acurácia K=9:", round(acc9, 3))
print("Acurácia K=5:", round(acc5, 3))

Acurácia K=9: 0.961
Acurácia K=5: 0.968


In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, knn5.predict(X_test), labels=['M','B'])
print(cm)

[[54  3]
 [ 2 95]]


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree1 = DecisionTreeClassifier(random_state=42)
tree1.fit(X_train, y_train)
y_pred1 = tree1.predict(X_test)
acc1 = accuracy_score(y_test, y_pred1)

tree2 = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=42)
tree2.fit(X_train, y_train)
y_pred2 = tree2.predict(X_test)
acc2 = accuracy_score(y_test, y_pred2)

print("Acurácia modelo padrão:", round(acc1, 3))
print("Acurácia modelo entropia + profundidade=4:", round(acc2, 3))

Acurácia modelo padrão: 0.942
Acurácia modelo entropia + profundidade=4: 0.974


In [ ]:
acc_train = accuracy_score(y_train, tree2.predict(X_train))
acc_test  = accuracy_score(y_test, tree2.predict(X_test))

print("Treino:", round(acc_train, 3))
print("Teste :", round(acc_test, 3))

Treino: 0.981
Teste : 0.974


In [ ]:
cols = X_train_raw.columns

X_new_raw = new_breast.drop(columns=['id', 'diagnosis'], errors='ignore')
X_new_raw = X_new_raw.reindex(columns=cols)

missing = set(cols) - set(X_new_raw.columns)
assert not missing, f"Faltando colunas em new_breast: {missing}"

X_new = pd.DataFrame(
    scaler.transform(X_new_raw),
    columns=cols,
    index=X_new_raw.index
)
y_new_pred = tree2.predict(X_new)

print(list(y_new_pred[:5]))

['B', 'M', 'B', 'B', 'B']


In [ ]:
import pandas as pd

pd.Series(y_new_pred).value_counts()

,count
B,17
M,10


# Exercício. Treine modelos Knn

Treine dois modelos Knn, um com k=9 e outro com k=5. Verifique a acuracidade de cada modelo para o conjunto de teste.





# Exercício. Treine modelos de Árvore de Decisão

Treine dois modelos de Árvore de Decisão. Em ambos empregue `random_state=42`. No primeiro, empregue os parâmetros padrão e, no segundo a seleção de nós por entropia e uma poda de 4 níveis da árvore.





# Exercício. Prevendo novos Casos

Empregue o melhor modelo obtido (maior acuracidade) para classificar os novos casos de `new_breast`.

